# 概要

このノートは、Qiitaの記事

* 15分でできる日本語Word2Vec(https://qiita.com/makaishi2/items/63b7986f6da93dc55edd)

を試したログです。

# 参考

* gensim document(https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec)

# TODO

* 学習対象のドキュメントをもっと増やす
* gensim関連のAPIドキュメント読んで中身を深掘り

# 本題

## 青空文庫より「三四郎」をダウンロードし整形

In [ ]:
!curl -LO http://www.aozora.gr.jp/cards/000148/files/794_ruby_4237.zip
!unzip 794_ruby_4237.zip
!ls -l sanshiro.txt

In [ ]:
import codecs
# ファイル読込み、内部表現化
f = codecs.open('sanshiro.txt', "r", "sjis")
text = f.read()
f.close()

# ファイル整形
import re
# ヘッダ部分の除去
text = re.split('\-{5,}',text)[2]
# フッタ部分の除去
text = re.split('底本：',text)[0]
# | の除去
text = text.replace('|', '')
# ルビの削除
text = re.sub('《.+?》', '', text)
# 入力注の削除
text = re.sub('［＃.+?］', '',text)
# 空行の削除
text = re.sub('\n\n', '\n', text) 
text = re.sub('\r', '', text)

# 整形結果確認

# 頭の100文字の表示 
print(text[:100])
# 見やすくするため、空行 
print()
print()
# 後ろの100文字の表示 
print(text[-100:])

In [ ]:
# Janomeのロード
from janome.tokenizer import Tokenizer

# Tokenneizerインスタンスの生成 
t = Tokenizer()

# テキストを引数として、形態素解析の結果、名詞・動詞原型のみを配列で抽出する関数を定義 
def extract_words(text):
    tokens = t.tokenize(text)
    return [token.base_form for token in tokens 
        if token.part_of_speech.split(',')[0] in['名詞', '動詞']]

#  関数テスト
ret = extract_words('三四郎は京都でちょっと用があって降りたついでに。')
for word in ret:
    print(word)

# 全体のテキストを句点('。')で区切った配列にする。 
sentences = text.split('。')
# それぞれの文章を単語リストに変換(処理に数分かかります)
word_list = [extract_words(sentence) for sentence in sentences]

# 結果の一部を確認 
for word in word_list[0]:
    print(word)

## 準備したデータを用いてWord2Vecのモデル作成、学習実施

In [ ]:
# Word2Vecライブラリのロード
from gensim.models import word2vec

# size: 圧縮次元数
# min_count: 出現頻度の低いものをカットする
# window: 前後の単語を拾う際の窓の広さを決める
# iter: 機械学習の繰り返し回数(デフォルト:5)十分学習できていないときにこの値を調整する
# model.wv.most_similarの結果が1に近いものばかりで、model.dict['wv']のベクトル値が小さい値ばかりの 
# ときは、学習回数が少ないと考えられます。
# その場合、iterの値を大きくして、再度学習を行います。

# 事前準備したword_listを使ってWord2Vecの学習実施
model = word2vec.Word2Vec(word_list, size=100,min_count=5,window=5,iter=100)

In [ ]:
# 学習ずみモデル保存
model.save("word2vec.model")

## 学習モデルを使い類似単語の調査

In [ ]:
# 学習済みモデル読み込み
saved_model = word2vec.Word2Vec.load("word2vec.model")

In [ ]:
# 結果の確認1
# 一つ一つの単語は100次元のベクトルになっています。 
# 「世間」のベクトル値を確認します。
print(saved_model.__dict__['wv']['世間'])

In [ ]:
# 結果の確認2
# 関数most_similarを使って「世間」の類似単語を調べます 
ret = saved_model.wv.most_similar(positive=['京都']) 
for item in ret:
    print(item[0], item[1])